In [57]:
# Imports
import random

import math

class Vertex:
    def __init__(self, key, x, y):
        self.key = key
        self.x = x
        self.y = y
        self.d = float('-inf')  # Distance from source
        self.h = None  # Heuristic distance to destination
        self.pi = None
        self.pi_set = set()
        self.priority = 0
        self.parent = None

    def __lt__(self, other):
        return self.priority < other.priority

    def __gt__(self, other):
        return self.priority > other.priority


class Graph:
    def __init__(self):
        self.vertices = {}
        self.adjacency_list = {}

    def add_vertex(self, key, x, y):
        if key not in self.vertices:
            self.vertices[key] = Vertex(key, x, y)
            self.adjacency_list[key] = []

    def add_edge(self, u, v):
        if u in self.vertices and v in self.vertices and u != v:
            if v not in self.adjacency_list[u]:
                self.adjacency_list[u].append(v)
            if u not in self.adjacency_list[v]:
                self.adjacency_list[v].append(u)  # For undirected graph


    def get_adjacency_list(self, vertex_key):
        if vertex_key in self.vertices:
            return self.adjacency_list[vertex_key]
        else:
            return []
    
    def max_degree(self):
        max_degree = 0
        for vertex_key in self.vertices.keys():
            degree = len(self.adjacency_list[vertex_key])
            max_degree = max(max_degree, degree)
        return max_degree
    
    def average_degree(self):
        total_degree = 0
        for vertex_key in self.vertices.keys():
            total_degree += len(self.adjacency_list[vertex_key])
        if len(self.vertices) > 0:
            return total_degree / len(self.vertices)
        else:
            return 0

    def generate_geometric_graph(self, n, r):

        for key in range(n):
            self.add_vertex(key,random.uniform(0, 1), random.uniform(0, 1))

        for u in self.vertices:
            for v in self.vertices:
                if u != v and euclidean_distance(self.vertices[u], self.vertices[v]) <= r:
                    self.adjacency_list[u].append(v)
                    self.adjacency_list[v].append(u)


def euclidean_distance(v1, v2):
    return math.sqrt(((v1.x - v2.x) ** 2) + ((v1.y - v2.y) ** 2))



n_values = [300, 400, 500]
min_ratios = [0.9, 0.8, 0.7]
max_ratios = [0.95, 0.9, 0.8]

graph = Graph()

def largest_connected_component(graph):
    def dfs(graph, visited, vertex, component):
        visited.add(vertex)
        component.append(vertex)
        for neighbor in graph.get_adjacency_list(vertex):
            if neighbor not in visited:
                dfs(graph, visited, neighbor, component)

    visited = set()
    largest_component = []
    for vertex in graph.vertices:
        if vertex not in visited:
            component = []
            dfs(graph, visited, vertex, component)
            if len(component) > len(largest_component):
                largest_component = component

    lcc_graph = Graph()
    for vertex_key in largest_component:
        vertex = graph.vertices[vertex_key]
        lcc_graph.add_vertex(vertex.key, vertex.x, vertex.y)
        for neighbor_key in graph.get_adjacency_list(vertex_key):
            if neighbor_key in largest_component:
                lcc_graph.add_edge(vertex.key, neighbor_key)

    return lcc_graph


def binary_search_r(n, min_ratio, max_ratio):
    low = 0.0
    high = 1.0
    while low <= high:
        graph = Graph()
        mid = (low + high) / 2
        vlcc_min = int(min_ratio * n)
        vlcc_max = int(max_ratio * n)
        graph.generate_geometric_graph(n, mid)
        lcc = largest_connected_component(graph)
        lcc_size = len(lcc.vertices)
        if vlcc_min <= lcc_size <= vlcc_max:
            return mid, lcc
        elif lcc_size > vlcc_min:
            high = mid - 0.00001
        else:
            low = mid + 0.00001
    return None, None  # No suitable r found


def store_graph_to_file(graph, filename):
    with open(filename, 'w') as f:
        processed_edges = set()  # To avoid duplicates
        for u in graph.vertices:
            for v in graph.get_adjacency_list(u):
                # Generate a unique key for the edge (u, v) by sorting the vertices' keys
                edge_key = (u, v) if u < v else (v, u)
                if edge_key not in processed_edges:
                    vertex_u = graph.vertices[u]
                    vertex_v = graph.vertices[v]
                    f.write(f"{u} {vertex_u.x} {vertex_u.y} {v} {vertex_v.x} {vertex_v.y}\n")
                    processed_edges.add(edge_key)


n_values = [300, 400, 500]
min_ratios = [0.9, 0.8, 0.7]
max_ratios = [0.95, 0.9, 0.8]

# for n, min_ratio, max_ratio in zip(n_values, min_ratios, max_ratios):
#     r, lcc = binary_search_r(n, min_ratio, max_ratio)
#     if r is not None:
#         store_graph_to_file(lcc, f"graph_n{n}_r{r}.EDGES")
#         print(f"Graph with n={n} and r={r} stored successfully.")
#     else:
#         print(f"No suitable r found for n={n}.")


def read_graph_from_file(filename):
    graph = Graph()
    with open(filename, 'r') as f:
        for line in f:
            data = line.split()
            u = int(data[0])
            x1 = float(data[1])
            y1 = float(data[2])
            v = int(data[3])
            x2 = float(data[4])
            y2 = float(data[5])

            if u not in graph.vertices:
                graph.add_vertex(u, x1, y1)
            if v not in graph.vertices:
                graph.add_vertex(v, x2, y2)

            graph.add_edge(u, v)

    return graph


graph = read_graph_from_file("graph_n300_r0.08202289062500001.EDGES")
# graph = read_graph_from_file("graph_n400_r0.06249125.EDGES")
# graph = read_graph_from_file("graph_n500_r0.05467859375.EDGES")

if 13 in graph.vertices:
    print("yes")


In [58]:
#  A star
import heapq

def initialize_single_source_max(graph, source, destination):
    for vertex in graph.vertices.values():
        vertex.d = float('-inf')
        vertex.h = euclidean_distance(vertex, destination)
        vertex.pi = None
        vertex.pi_set = set()

    source.d = 0

def relax_max(u, v):
    if v.d < (u.d + 1):
        v.d = u.d + 1
        v.pi = u
        return True
    return False

def a_star_max(graph, source, destination):
    initialize_single_source_max(graph, source, destination)
    closed_set = set()
    open_heap = []
    heapq.heappush(open_heap, (-source.d - source.h, source))
    
    while open_heap:
        _, u = heapq.heappop(open_heap)
        closed_set.add(u)

        if u == destination:
            break

        for v_key in graph.get_adjacency_list(u.key):
            v = graph.vertices[v_key]
            if v in closed_set:
                continue
            if relax_max(u, v):
                v.priority = v.d + v.h
                heapq.heappush(open_heap, (-v.priority, v))

final_path = []

for i in range(0, len(graph.vertices)):
    if i in graph.vertices:
        for j in range(0, len(graph.vertices)):
            if i != j and j in graph.vertices:
                s = graph.vertices[i]
                d = graph.vertices[j]
                a_star_max(graph, s, d)
                path = []
                x = d
                if x.d != float('-inf'):
                    visited = {x}
                    while x != s:
                        path.insert(0, x.key)
                        visited.add(x)
                        x = x.pi
                        if x in visited:
                            break
                    path.insert(0, s.key)
                    if (len(path) > len(final_path)):
                        final_path = path

final_path


print(final_path)
print(len(final_path))  

[178, 90, 275, 76, 132, 278, 207, 230, 189, 64, 292, 126, 17, 23, 222, 285, 137, 224, 150, 24, 214, 155, 235, 205, 252, 101, 121, 288, 255, 180, 160, 106, 227, 84, 282, 165, 264, 139, 53, 212, 5, 236, 123, 25, 182, 256, 63, 228, 98, 11, 77, 38, 263, 268, 105, 193, 72, 110, 188, 20, 55, 158, 6, 173, 35, 129, 8, 130, 151, 237, 262, 217, 290, 161, 168, 284, 12, 147, 149, 167, 118, 127, 170, 108, 14, 297, 184, 163, 148, 95, 112, 143, 238, 89, 223, 94, 156, 86, 159, 45, 266, 277, 28, 211, 117, 88, 125, 59, 107, 225, 140, 194, 100, 259, 289, 253, 31, 36, 243, 215, 287, 181, 274, 213, 294, 281, 15, 251, 240, 21, 47, 7, 152, 61, 226, 144, 206, 103, 208, 34, 102, 183, 299, 56, 83, 296, 234, 279, 291, 185, 229, 44, 16, 166, 81, 254, 113, 115, 62, 248, 242, 3, 92, 85, 190, 49, 65, 96, 271, 265, 249, 202, 283, 136, 231, 241, 116, 9, 233, 10, 93, 201, 82, 42, 146, 153, 171, 246, 52, 19, 58, 109, 220, 40, 66, 192, 78, 204, 164, 104, 41, 50, 270, 295, 97, 276, 67, 135, 51, 203, 195, 57, 145, 43, 120,

In [62]:
# DFS

def DFSAlgo(graph):
    def dfs(node, visited, path, memo):
        nonlocal max_length, longest_path
        visited[node] = True
        path.append(node)

        for neighbor in graph.adjacency_list[node]:
            if not visited[neighbor]:
                if neighbor not in memo or len(memo[neighbor]) < len(path):
                    dfs(neighbor, visited, path, memo)

        if len(path) > max_length:
            max_length = len(path)
            longest_path = path.copy()

        path.pop()
        visited[node] = False
        memo[node] = path.copy()

    max_length = float('-inf')
    longest_path = []

    visited = {v: False for v in graph.vertices}
    memo = {}

    for vertex in graph.vertices:
        if not visited[vertex]:
            dfs(vertex, visited, [], memo)

    return longest_path

a = DFSAlgo(graph)

print(a)
print(len(a))


[297, 108, 14, 127, 118, 167, 149, 12, 147, 170, 161, 290, 60, 87, 217, 223, 94, 89, 262, 237, 28, 86, 130, 8, 129, 35, 6, 173, 158, 55, 203, 20, 188, 135, 51, 110, 72, 193, 105, 53, 139, 264, 160, 180, 288, 252, 101, 121, 255, 123, 5, 25, 235, 182, 214, 24, 11, 267, 19, 58, 104, 41, 50, 270, 97, 43, 120, 293, 216, 273, 128, 46, 79, 27, 152, 61, 117, 88, 238, 95, 112, 59, 107, 140, 31, 36, 100, 15, 34, 47, 21, 102, 56, 83, 296, 279, 30, 177, 116, 80, 233, 210, 197, 10, 111, 9, 0, 91, 32, 298, 4, 44, 16, 96, 271, 70, 172, 185, 291, 299, 183, 74, 71, 181, 213, 215, 194, 240, 243, 251, 281, 253, 259, 287, 274, 289]
136
